In [2]:
import sys

sys.path.append('../')

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn import preprocessing


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#transformers
from transformers import BertTokenizer
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

# our functions
from src.data.make_dataset import final_dataset
from src.data.fix_balance import sampling_balance
from src.data.split_data import split
from src.data.encode_data import ohe_encoding

#set seed for reproducibility
SEED=42

/home/nfrvnikita/projects/service4classification/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-24 03:09:41.917145: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-24 03:09:41.918486: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 03:09:41.937259: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-24 03:09:41.937293: E tensorflow/compiler/xla/stream_executor

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


2023-08-24 03:09:42.473491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
data = pd.read_csv('/home/nfrvnikita/projects/service4classification/data/raw/BRFRD.csv')
data.head()

,review,rating,published,full review url
0,"«Головоломка» — продукт универсальный, и, чтоб...",90,2015-6-22,http://www.kino-mir.ru/posts/view/147
1,"А если подвести итоги, то Пиксар создал, не по...",100,2015-6-18,https://www.uralweb.ru/poster/reviews/6694.html
2,И все же плюсы «Головоломки» перевешивают ее м...,100,2015-6-19,http://www.tramvision.ru/recensia/2015/golovol...
3,"На выходе из зала есть ощущение, что Pixar сде...",100,2015-6-18,http://afisha.ngs.ru/news/more/2181412/
4,"Да, перед нами настоящий, старой школы пиксаро...",100,2015-6-16,http://www.kinokadr.ru/articles/2015/06/17/ins...


#### Генерация обработанного датасета

In [5]:
final_data = final_dataset(data)

/home/nfrvnikita/projects/service4classification/notebooks/../src/features/build_features.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['token_lens'] = token_lens


In [6]:
final_data.head()

,review,rating,clean_review,text_len,token_lens,class
0,История о любви и простом человеческом сострад...,80,история о любви и простом человеческом сострад...,36,73,2
1,После сериала «Родина» на режиссера Лунгина па...,70,после сериала родина на режиссера лунгина пала...,36,69,1
2,"Фильм, действительно, получился вдохновенным. ...",80,фильм действительно получился вдохновенным что...,34,61,2
3,«Однажды в Трубчевске» раскрывает сюжет в усло...,80,однажды в трубчевске раскрывает сюжет в услови...,38,71,2
4,Важнейшим достоинством фильма является соверше...,80,важнейшим достоинством фильма является соверше...,41,59,2


In [7]:
final_data['class'].value_counts()

class
2    15526
1     9706
0     2063
Name: count, dtype: int64

#### Дисбаланс классов

In [8]:
data_os = sampling_balance(final_data)

In [9]:
data_os['class'].value_counts()

class
2    15526
1    15526
0    15526
Name: count, dtype: int64

#### Сплит данных на трейн и валид датасеты

In [10]:
X_train, X_valid, y_train, y_valid = split(data_os)

#### OneHotEncoding

In [11]:
y_train, y_valid = ohe_encoding(y_train, y_valid)

#### Векторизация (TF-IDF, CountVectorizer)